## stacking 集成算法：生成次级学习器数据集
基础学习器抽取特征，次级学习器融合。5折。

In [1]:
# 导入必要的工具包
from sklearn.model_selection import train_test_split
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写

In [2]:
path_train_all = '../data/new/train=all/' #训练集，特征工程后的文件 
path_train = '../data/new/train=predict/' #训练集，预测，得到的特征
train_name = '%strain=pred-%s-%s.csv' #训练集，预测特征，文件名
path_test = '../data/new/test=predict/' #测试集，预测，得到的特征
test_name = '%stest=pred-%s-%s.csv' #测试集，预测特征，文件名 

val_path = '../data/val/' #保存校验集测试结果
val_name = '%sval=stacking-base2-%s-%s.csv' #校验集预测结果文件名
summit_path = '../data/summit/stacking2/' #保存提交文件，测试集测试结果 
out_name = '%ssummit=stacking-base2-%s-%s.csv' # 生成的结果文件名称

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 1.读取校验集预测结果，0,1,2,3,4合并。

In [3]:
# # model_name = 'RidgeCV'
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df = pd.read_csv(val_name % (val_path, model_name, i))
#     else:
#         col_df = pd.concat([col_df, pd.read_csv(val_name % (val_path, model_name, i))])
#     print('col_df.shape=', col_df.shape)
# col_df.sort_values(user_id, inplace=True)
# col_df.head()

In [4]:
# val=stacking-base-RidgeCV-0
model_list = ['RidgeCV', 'RandomForestRegressor','lasso','GradientBoostingRegressor','ExtraTreesRegressor',
              'ElasticNetCV','DNN_V2', 'BaggingRegressor']
for j, model_name in enumerate(model_list): #每个模型预测一个特征 
#     print('j=%s model_name=%s' % (j, model_name))
    for i in range(train_part_num): # 读取校验集预测结果，0,1,2,3,4合并。
        if i == 0:
            col_df = pd.read_csv(val_name % (val_path, model_name, i))
        else:
            col_df = pd.concat([col_df, pd.read_csv(val_name % (val_path, model_name, i))])
#         print('col_df.shape=', col_df.shape)
    col_df.sort_values(user_id, inplace=True) #必须排序保持一致 
    if j == 0:
        train_df = pd.DataFrame(col_df[user_id], columns=[user_id], dtype=np.int32)
    train_df['pred_%s'%j] = col_df[target].astype(np.float32) #多个特征合并，得到整个数据集。
    print('train_df.shape=', train_df.shape)

train_df.shape= (2288007, 2)
train_df.shape= (2288007, 3)
train_df.shape= (2288007, 4)
train_df.shape= (2288007, 5)
train_df.shape= (2288007, 6)
train_df.shape= (2288007, 7)
train_df.shape= (2288007, 8)
train_df.shape= (2288007, 9)


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288007 entries, 218050 to 86498
Data columns (total 9 columns):
user_id    int32
pred_0     float32
pred_1     float32
pred_2     float32
pred_3     float32
pred_4     float32
pred_5     float32
pred_6     float32
pred_7     float32
dtypes: float32(8), int32(1)
memory usage: 96.0 MB


In [6]:
# 预测特征 v1 特征个数 =9
train_df.to_csv(train_name %(path_train, '1', 'col=9'), index=False)
train_df.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
218050,1,0.000000,0.000056,0.0,0.484938,0.004374,0.0,0.622804,0.000059
99015,1593,0.000000,0.052974,0.0,0.484938,0.042844,0.0,0.452778,0.036465
282730,1594,0.194275,0.000119,0.0,0.349532,0.004415,0.0,0.083804,0.000068
112115,1595,0.045743,0.000311,0.0,0.413887,0.000075,0.0,0.277812,0.000298
298802,1596,0.045501,0.000064,0.0,0.442441,0.001589,0.0,0.276851,0.000137


## 2.读取测试集预测结果，0,1,2,3,4平均。

In [7]:
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
#         col_df1.sort_values(user_id, inplace=True) #必须排序保持一致 
#     else:
#         col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
#         col_df2.sort_values(user_id, inplace=True) #必须排序保持一致 
# #         print('i=', i)
# #         print(col_df1.head())
# #         print(col_df2.head())
#         col_df1[target] = (col_df1[target] + col_df2[target]) / 2
# #         print(col_df1.head())
# col_df1.head() #两种方式结果不同 

In [8]:
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
#     else:
#         col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
# #         print('i=', i)
# #         print(col_df1.head())
# #         print(col_df2.head())
#         col_df1[target] = (col_df1[target] + col_df2[target])
# #         print(col_df1.head())
# col_df1[target] = (col_df1[target] / train_part_num)
# col_df1.head() #这种方式更好 

In [9]:
model_list = ['RidgeCV', 'RandomForestRegressor','lasso','GradientBoostingRegressor','ExtraTreesRegressor',
              'ElasticNetCV','DNN_V2', 'BaggingRegressor']
for j, model_name in enumerate(model_list): #每个模型预测一个特征 
#     print('j=%s model_name=%s' % (j, model_name))
    for i in range(train_part_num):
        if i == 0:
            col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
        else:
            col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
#             print('i=', i)
#             print(col_df1.head())
#             print(col_df2.head())
            col_df1[target] = (col_df1[target] + col_df2[target])
#             print(col_df1.head())
    col_df1[target] = (col_df1[target] / train_part_num) #求平均
    if j == 0:
        test_df = pd.DataFrame(col_df1[user_id], columns=[user_id], dtype=np.int32)
    test_df['pred_%s'%j] = col_df1[target].astype(np.float32) #多个特征合并，得到整个数据集。
    print('test_df.shape=', test_df.shape)

test_df.shape= (828934, 2)
test_df.shape= (828934, 3)
test_df.shape= (828934, 4)
test_df.shape= (828934, 5)
test_df.shape= (828934, 6)
test_df.shape= (828934, 7)
test_df.shape= (828934, 8)
test_df.shape= (828934, 9)


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828934 entries, 0 to 828933
Data columns (total 9 columns):
user_id    828934 non-null int32
pred_0     828934 non-null float32
pred_1     828934 non-null float32
pred_2     828934 non-null float32
pred_3     828934 non-null float32
pred_4     828934 non-null float32
pred_5     828934 non-null float32
pred_6     828934 non-null float32
pred_7     828934 non-null float32
dtypes: float32(8), int32(1)
memory usage: 28.5 MB


In [11]:
# 预测特征 v1 特征个数 =9
test_df.to_csv(test_name %(path_test, '1', 'col=9'), index=False)
test_df.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,14933,0.084183,0.000180,0.007868,0.406852,1.084812e-04,0.010231,1.650189,0.000162
1,14934,0.059209,0.000117,0.000000,0.406852,2.858160e-05,0.000000,0.671875,0.000123
2,14935,0.392662,0.000134,0.000000,0.406852,4.165320e-05,0.000000,0.931189,0.000121
3,14936,1.513114,0.006305,0.235470,0.406852,1.391971e-08,0.235131,1.364640,0.000070
4,14937,0.000000,0.000141,0.000000,0.406852,3.695815e-05,0.000000,0.599589,0.000145


## 3.给训练集添加 target 
### 3.1 首先读取特征工程后的训练集 ，指定读取数据类型，节省内存

In [12]:
col_int32 = ['user_id', 'register_time', 'wood_add_value', 'wood_reduce_value',
       'stone_add_value', 'stone_reduce_value', 'ivory_add_value',
       'ivory_reduce_value', 'meat_add_value', 'meat_reduce_value',
       'magic_add_value', 'magic_reduce_value', 'infantry_add_value',
       'infantry_reduce_value', 'cavalry_add_value', 'cavalry_reduce_value',
       'shaman_add_value', 'shaman_reduce_value', 'wound_infantry_add_value',
       'wound_infantry_reduce_value', 'wound_cavalry_add_value',
       'wound_cavalry_reduce_value', 'wound_shaman_add_value',
       'wound_shaman_reduce_value', 'general_acceleration_add_value',
       'general_acceleration_reduce_value', 'building_acceleration_add_value',
       'building_acceleration_reduce_value', 'reaserch_acceleration_add_value',
       'reaserch_acceleration_reduce_value', 'training_acceleration_add_value',
       'training_acceleration_reduce_value', 'treatment_acceleraion_add_value',
       'treatment_acceleration_reduce_value', 'bd_training_hut_level',
       'bd_healing_lodge_level', 'bd_stronghold_level',
       'bd_outpost_portal_level', 'bd_barrack_level',
       'bd_healing_spring_level', 'bd_dolmen_level', 'bd_guest_cavern_level',
       'bd_warehouse_level', 'bd_watchtower_level', 'bd_magic_coin_tree_level',
       'bd_hall_of_war_level', 'bd_market_level', 'bd_hero_gacha_level',
       'bd_hero_strengthen_level', 'bd_hero_pve_level', 'sr_scout_level', 
       'sr_training_speed_level', 'sr_infantry_tier_2_level',
       'sr_cavalry_tier_2_level', 'sr_shaman_tier_2_level',
       'sr_infantry_atk_level', 'sr_cavalry_atk_level', 'sr_shaman_atk_level',
       'sr_infantry_tier_3_level', 'sr_cavalry_tier_3_level',
       'sr_shaman_tier_3_level', 'sr_troop_defense_level',
       'sr_infantry_def_level', 'sr_cavalry_def_level', 'sr_shaman_def_level',
       'sr_infantry_hp_level', 'sr_cavalry_hp_level', 'sr_shaman_hp_level',
       'sr_infantry_tier_4_level', 'sr_cavalry_tier_4_level',
       'sr_shaman_tier_4_level', 'sr_troop_attack_level',
       'sr_construction_speed_level', 'sr_hide_storage_level',
       'sr_troop_consumption_level', 'sr_rss_a_prod_levell',
       'sr_rss_b_prod_level', 'sr_rss_c_prod_level', 'sr_rss_d_prod_level',
       'sr_rss_a_gather_level', 'sr_rss_b_gather_level',
       'sr_rss_c_gather_level', 'sr_rss_d_gather_level', 'sr_troop_load_level',
       'sr_rss_e_gather_level', 'sr_rss_e_prod_level',
       'sr_outpost_durability_level', 'sr_outpost_tier_2_level',
       'sr_healing_space_level', 'sr_gathering_hunter_buff_level',
       'sr_healing_speed_level', 'sr_outpost_tier_3_level',
       'sr_alliance_march_speed_level', 'sr_pvp_march_speed_level',
       'sr_gathering_march_speed_level', 'sr_outpost_tier_4_level',
       'sr_guest_troop_capacity_level', 'sr_march_size_level',
       'sr_rss_help_bonus_level', 'pvp_battle_count', 'pvp_lanch_count',
       'pvp_win_count', 'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'pay_count']
col_float32 = ['avg_online_minutes', 'pay_price']

In [13]:
# 指定读取数据类型，节省内存
dtype_test = {}
for col in col_int32:
    dtype_test[col] = np.int32
for col in col_float32:
    dtype_test[col] = np.float32
# dtype_test

In [14]:
dtype_train = dtype_test.copy()
dtype_train[target] = np.float32
# dtype_train

In [15]:
train_fe = pd.read_csv('%strain=fe.csv' % (path_train_all), dtype=dtype_train)
train_fe.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,1,128,20125,3700,0,0,0,0,16375,2000,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
1,1593,136,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
2,1594,136,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.166667,0.0,0,0.0
3,1595,136,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.166667,0.0,0,0.0
4,1596,136,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2.333333,0.0,0,0.0


### 3.2 然后只取 userid 和 target

In [16]:
train_fe2 = train_fe[[user_id, 'prediction_pay_price']]
train_target_name = '%strain=fe-%s.csv' % (path_train_all, 'target')
train_fe2.to_csv(train_target_name, index=False)
train_fe2.head()

,user_id,prediction_pay_price
0,1,0.0
1,1593,0.0
2,1594,0.0
3,1595,0.0
4,1596,0.0


In [17]:
# 看看有没有不同 id 
# print(train_fe2[user_id].value_counts())
# print(train_pred[user_id].value_counts())

### 3.3 把 target 添加到 pred 里面

In [18]:
train_pred = pd.read_csv(train_name %(path_train, '1', 'col=9'))
train_pred.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,1,0.000000,0.000056,0.0,0.484938,0.004374,0.0,0.622804,0.000059
1,1593,0.000000,0.052974,0.0,0.484938,0.042844,0.0,0.452778,0.036465
2,1594,0.194275,0.000119,0.0,0.349532,0.004415,0.0,0.083804,0.000068
3,1595,0.045743,0.000311,0.0,0.413887,0.000075,0.0,0.277812,0.000298
4,1596,0.045501,0.000064,0.0,0.442441,0.001589,0.0,0.276851,0.000137


In [19]:
train_pred[target] = train_fe2[target]
train_pred.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,prediction_pay_price
0,1,0.000000,0.000056,0.0,0.484938,0.004374,0.0,0.622804,0.000059,0.0
1,1593,0.000000,0.052974,0.0,0.484938,0.042844,0.0,0.452778,0.036465,0.0
2,1594,0.194275,0.000119,0.0,0.349532,0.004415,0.0,0.083804,0.000068,0.0
3,1595,0.045743,0.000311,0.0,0.413887,0.000075,0.0,0.277812,0.000298,0.0
4,1596,0.045501,0.000064,0.0,0.442441,0.001589,0.0,0.276851,0.000137,0.0


In [20]:
train_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288007 entries, 0 to 2288006
Data columns (total 10 columns):
user_id                 int64
pred_0                  float64
pred_1                  float64
pred_2                  float64
pred_3                  float64
pred_4                  float64
pred_5                  float64
pred_6                  float64
pred_7                  float64
prediction_pay_price    float32
dtypes: float32(1), float64(8), int64(1)
memory usage: 165.8 MB


### 3.4 把 64 位数据转换为 32 位数据，节省内存

In [21]:
col_int64 = train_pred.select_dtypes(include = ["int64"]).columns
col_int64.shape

(1,)

In [22]:
col_float64 = train_pred.select_dtypes(include = ["float64"]).columns
col_float64.shape

(8,)

In [23]:
train_pred[col_int64] = train_pred[col_int64].astype(np.int32)
train_pred[col_float64] = train_pred[col_float64].astype(np.float32)
train_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288007 entries, 0 to 2288006
Data columns (total 10 columns):
user_id                 int32
pred_0                  float32
pred_1                  float32
pred_2                  float32
pred_3                  float32
pred_4                  float32
pred_5                  float32
pred_6                  float32
pred_7                  float32
prediction_pay_price    float32
dtypes: float32(9), int32(1)
memory usage: 87.3 MB


In [24]:
# 保存文件
train_pred.to_csv(train_name %(path_train, '1', 'col=9-target'), index=False)
train_pred.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,prediction_pay_price
0,1,0.000000,0.000056,0.0,0.484938,0.004374,0.0,0.622804,0.000059,0.0
1,1593,0.000000,0.052974,0.0,0.484938,0.042844,0.0,0.452778,0.036465,0.0
2,1594,0.194275,0.000119,0.0,0.349532,0.004415,0.0,0.083804,0.000068,0.0
3,1595,0.045743,0.000311,0.0,0.413887,0.000075,0.0,0.277812,0.000298,0.0
4,1596,0.045501,0.000064,0.0,0.442441,0.001589,0.0,0.276851,0.000137,0.0


## 4. 拆分为 5 份，随机抽样

In [25]:
# 先取出 0.2 剩余的 0.8 再平分为 0.4 再平分为 0.2 
train_part2345, train_part1 = train_test_split(train_pred, random_state=33, test_size=0.2)
print('train.shape=%s train_part2345.shape=%s train_part1.shape=%s'
      %(train_pred.shape, train_part2345.shape, train_part1.shape))

train.shape=(2288007, 10) train_part2345.shape=(1830405, 10) train_part1.shape=(457602, 10)


In [26]:
# train=fe-1.csv 特征工程后的，注册日期转换为天数
train_part1.to_csv(train_name %(path_train, '1', 'col=9-target-1'), index=False)
train_part1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457602 entries, 295995 to 525208
Data columns (total 10 columns):
user_id                 457602 non-null int32
pred_0                  457602 non-null float32
pred_1                  457602 non-null float32
pred_2                  457602 non-null float32
pred_3                  457602 non-null float32
pred_4                  457602 non-null float32
pred_5                  457602 non-null float32
pred_6                  457602 non-null float32
pred_7                  457602 non-null float32
prediction_pay_price    457602 non-null float32
dtypes: float32(9), int32(1)
memory usage: 20.9 MB


In [27]:
train_part23, train_part45 = train_test_split(train_part2345, random_state=33, test_size=0.5)
print('train_part2345.shape=%s train_part23.shape=%s train_part45.shape=%s'
      %(train_part2345.shape, train_part23.shape, train_part45.shape))

train_part2, train_part3 = train_test_split(train_part23, random_state=33, test_size=0.5)
print('train_part23.shape=%s train_part2.shape=%s train_part3.shape=%s'
      %(train_part23.shape, train_part2.shape, train_part3.shape))
train_part2.to_csv(train_name %(path_train, '1', 'col=9-target-2'), index=False)
train_part3.to_csv(train_name %(path_train, '1', 'col=9-target-3'), index=False)

train_part4, train_part5 = train_test_split(train_part45, random_state=33, test_size=0.5)
print('train_part45.shape=%s train_part4.shape=%s train_part5.shape=%s'
      %(train_part45.shape, train_part4.shape, train_part5.shape))
train_part4.to_csv(train_name %(path_train, '1', 'col=9-target-4'), index=False)
train_part5.to_csv(train_name %(path_train, '1', 'col=9-target-5'), index=False)

train_part2345.shape=(1830405, 10) train_part23.shape=(915202, 10) train_part45.shape=(915203, 10)
train_part23.shape=(915202, 10) train_part2.shape=(457601, 10) train_part3.shape=(457601, 10)
train_part45.shape=(915203, 10) train_part4.shape=(457601, 10) train_part5.shape=(457602, 10)
